<a href="https://colab.research.google.com/github/Johansmm/RT-multiperson-pose-pytorch/blob/imt-atlantique/pose_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning project
Intro by Tatiana

In [1]:
# Libraries
# Colab libraries
from google.colab import drive
from google.colab import output
drive.mount('/content/gdrive')
colab_path = "/content/gdrive/My Drive/Colab Notebooks/"

# Basis libraries
import os, re, sys, math, time, scipy, argparse
import cv2, matplotlib
import numpy as np
import pylab as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from collections import OrderedDict
from scipy.ndimage.morphology import generate_binary_structure
from scipy.ndimage.filters import gaussian_filter, maximum_filter

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Section 0: Download and install repository
First, we will download the repository that we copied from the [original repository](https://github.com/tensorboy/pytorch_Realtime_Multi-Person_Pose_Estimation) in order to make some changes for educational purposes. Then, we install libraries and some dependences explained in the original repository.

In [2]:
# Independence install
!sudo apt-get install swig
%cd $colab_path
if not os.path.isdir("RT-multiperson-pose-pytorch"):
  # Repository clone
  !git clone https://github.com/Johansmm/RT-multiperson-pose-pytorch.git
  %cd "RT-multiperson-pose-pytorch"
  %cd lib/pafprocess 
  # Repository compile
  !sh make.sh

# Libraries install
!python -m pip install -r ./requirements.txt
!git submodule init && git submodule update
%cd $colab_path"/RT-multiperson-pose-pytorch"
# Weights download
if not os.path.isfile("./pose_model.pth"):
  !wget https://www.dropbox.com/s/ae071mfm2qoyc8v/pose_model.pth
output.clear()
print("[INFO]: Proyect uploaded successfully")

[INFO]: Proyect uploaded successfully


In [3]:
!python demo/picture_demo.py

Bulding VGG19
0.5459940652818991


In [4]:
# Framework libraries
%cd $colab_path"/RT-multiperson-pose-pytorch"
sys.path.append('.')
from lib.network.rtpose_vgg import get_model
from lib.network import im_transform
from lib.utils.common import Human, BodyPart, CocoPart, CocoColors, CocoPairsRender, draw_humans
from lib.utils.paf_to_pose import paf_to_pose_cpp
from lib.config import cfg, update_config
from evaluate.coco_eval import get_outputs, handle_paf_and_heat
data_download = False # For download COCO dataset

/content/gdrive/My Drive/Colab Notebooks/RT-multiperson-pose-pytorch


usage: ipykernel_launcher.py [-h] [--cfg CFG] [--weight WEIGHT] ...
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Section 2: Replication of results
In this section we replicated some resutls. First, we need download the data. For this case, we will use the `sh` compiler provided by [original repository](https://github.com/tensorboy/pytorch_Realtime_Multi-Person_Pose_Estimation)

In [ ]:
%cd $colab_path"/RT-multiperson-pose-pytorch"
if data_download and not os.path.isdir("data/coco"):
  !mkdir data
  %cd data
  !sh ../lib/datasets/CocoDataDownloader.sh
  %cd $colab_path"/RT-multiperson-pose-pytorch"
  output.clear()
  print("[INFO]: Coco database downloaded successfully")

Now, we defined some principal functions.

In [ ]:
%cd $colab_path"/RT-multiperson-pose-pytorch"
class Namespace:
  def __init__(self, **kwargs):
    self.__dict__.update(kwargs)

def load_rtpose_model():
  args = Namespace(cfg = './experiments/vgg19_368x368_sgd.yaml', weight = 'pose_model.pth', opts = [])
  update_config(cfg, args)
  model = get_model('vgg19')     
  model.load_state_dict(torch.load(args.weight))
  model = torch.nn.DataParallel(model).cuda()
  model.float()
  model.eval()
  return model

def im_forward(image, model):
  with torch.no_grad():
    paf, heatmap, im_scale = get_outputs(image, model, 'rtpose')
  return paf, heatmap, im_scale

model = load_rtpose_model()

In [ ]:
image = cv2.imread('./data/coco/images/training2017/000000000154.jpg')
paf, heatmap, im_scale = im_forward(image, model)
humans = paf_to_pose_cpp(heatmap, paf, cfg)
for hum in humans:
  print(dir(hum))

out = draw_humans(image, humans)
plt.imshow(out)


In [18]:
# Colab libraries
from google.colab import drive
from google.colab import output
drive.mount('/content/gdrive')

%cd "/content/gdrive/My Drive/Colab Notebooks/RT-multiperson-pose-pytorch"
!git config --global user.email "johan-steven.mejia-mogollon@imt-atlantique.com"
!git config --global user.name "Johansmm"
!git remote set-url origin "https://github.com/Johansmm/RT-multiperson-pose-pytorch.git"
# !git commit -am "updating in colab"
# !git reset --hard
!git pull origin imt-atlantique
# !git push origin imt-atlantique

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/RT-multiperson-pose-pytorch
On branch master
Your branch is ahead of 'origin/master' by 24 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
From https://github.com/Johansmm/RT-multiperson-pose-pytorch
 * branch            imt-atlantique -> FETCH_HEAD
Already up to date.
